GOAL2: Use the cleaned dataset with Pandas to perform EDA 

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy as sp
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings('ignore')

For the EDA part of the project we will use Pandas to handle the already cleaned dataset

In [3]:
df = pd.read_csv("../Clean_Data/2024/clean_dataset_legislative_2024.csv", sep=";")

In [4]:
df.head(5)

,Code_département,Libellé_département,Code_commune,Libellé_commune,Inscrits,Votants,Pourcentage_Votants,Abstentions,Pourcentage_Abstentions,Exprimés,...,Numéro_de_panneau_18,Nuance_candidat_18,Nom_candidat_18,Prénom_candidat_18,Sexe_candidat_18,Voix_18,Pourcentage_Voix/inscrits_18,Pourcentage_Voix/exprimés_18,Elu_18,Nombre_candidats
0,1,Ain,1001,L'Abergement-Clémenciat,662,494,74.62,168,25.38,476,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2
1,1,Ain,1002,L'Abergement-de-Varey,228,187,82.02,41,17.98,166,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2
2,1,Ain,1004,Ambérieu-en-Bugey,8745,5887,67.32,2858,32.68,5348,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2
3,1,Ain,1005,Ambérieux-en-Dombes,1337,979,73.22,358,26.78,926,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2
4,1,Ain,1006,Ambléon,98,65,66.33,33,33.67,59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2
